In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      

In [ ]:
from datasets import load_dataset, load_dataset_builder, get_dataset_config_names, get_dataset_split_names
import pandas as pd
import string
import nltk
import spacy

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk import pos_tag

from collections import defaultdict
import re
import unicodedata
import random
from nltk.probability import ConditionalFreqDist

# **Preprocessing**

Get Dataset

In [ ]:
ds_builder = load_dataset_builder("squad_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_data = load_dataset("squad_v2", split="train")

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
train = pd.DataFrame(train_data)

In [ ]:
para = train['context'][200]
para

'At the 52nd Annual Grammy Awards, Beyoncé received ten nominations, including Album of the Year for I Am... Sasha Fierce, Record of the Year for "Halo", and Song of the Year for "Single Ladies (Put a Ring on It)", among others. She tied with Lauryn Hill for most Grammy nominations in a single year by a female artist. In 2010, Beyoncé was featured on Lady Gaga\'s single "Telephone" and its music video. The song topped the US Pop Songs chart, becoming the sixth number-one for both Beyoncé and Gaga, tying them with Mariah Carey for most number-ones since the Nielsen Top 40 airplay chart launched in 1992. "Telephone" received a Grammy Award nomination for Best Pop Collaboration with Vocals.'

Preprocessing

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
para = para.lower()
sent = nltk.sent_tokenize(para)

remove_digits = str.maketrans('', '',string.digits)
remove_punctuation = str.maketrans('', '', string.punctuation)
for i in range(len(sent)):
    sent[i] = sent[i].translate(remove_digits)
    sent[i] = sent[i].translate(remove_punctuation)

for i in range(len(sent)):
    sent[i] = word_tokenize(sent[i])

In [ ]:
sent[0]

['at',
 'the',
 'nd',
 'annual',
 'grammy',
 'awards',
 'beyoncé',
 'received',
 'ten',
 'nominations',
 'including',
 'album',
 'of',
 'the',
 'year',
 'for',
 'i',
 'am',
 'sasha',
 'fierce',
 'record',
 'of',
 'the',
 'year',
 'for',
 'halo',
 'and',
 'song',
 'of',
 'the',
 'year',
 'for',
 'single',
 'ladies',
 'put',
 'a',
 'ring',
 'on',
 'it',
 'among',
 'others']

# **Part of Speech Tagging**

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
tagged = []

for i in sent:
  tagged.append(pos_tag(i))

In [ ]:
tagged

[[('at', 'IN'),
  ('the', 'DT'),
  ('nd', 'JJ'),
  ('annual', 'JJ'),
  ('grammy', 'NN'),
  ('awards', 'NNS'),
  ('beyoncé', 'VBP'),
  ('received', 'VBN'),
  ('ten', 'JJ'),
  ('nominations', 'NNS'),
  ('including', 'VBG'),
  ('album', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('year', 'NN'),
  ('for', 'IN'),
  ('i', 'NN'),
  ('am', 'VBP'),
  ('sasha', 'JJ'),
  ('fierce', 'NN'),
  ('record', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('year', 'NN'),
  ('for', 'IN'),
  ('halo', 'NN'),
  ('and', 'CC'),
  ('song', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('year', 'NN'),
  ('for', 'IN'),
  ('single', 'JJ'),
  ('ladies', 'NNS'),
  ('put', 'VBD'),
  ('a', 'DT'),
  ('ring', 'NN'),
  ('on', 'IN'),
  ('it', 'PRP'),
  ('among', 'IN'),
  ('others', 'NNS')],
 [('she', 'PRP'),
  ('tied', 'VBD'),
  ('with', 'IN'),
  ('lauryn', 'NN'),
  ('hill', 'NN'),
  ('for', 'IN'),
  ('most', 'JJS'),
  ('grammy', 'JJ'),
  ('nominations', 'NNS'),
  ('in', 'IN'),
  ('a', 'DT'),
  ('single', 'JJ'),
  ('year', 'NN'),
 

In [ ]:
corpus = para
corpus

'at the 52nd annual grammy awards, beyoncé received ten nominations, including album of the year for i am... sasha fierce, record of the year for "halo", and song of the year for "single ladies (put a ring on it)", among others. she tied with lauryn hill for most grammy nominations in a single year by a female artist. in 2010, beyoncé was featured on lady gaga\'s single "telephone" and its music video. the song topped the us pop songs chart, becoming the sixth number-one for both beyoncé and gaga, tying them with mariah carey for most number-ones since the nielsen top 40 airplay chart launched in 1992. "telephone" received a grammy award nomination for best pop collaboration with vocals.'

# Bigram

Get Initial probabailities from corpus

In [ ]:
def build_conditional_probabilities(corpus):
	tokenized_string = corpus.split()
	previous_word = ""
	dictionnary = defaultdict(list)

	for current_word in tokenized_string:
		if previous_word != "":
			dictionnary[previous_word].append(current_word)
		previous_word = current_word

	for key in dictionnary.keys():
		next_words = dictionnary[key]
		unique_words = set(next_words)
		nb_words = len(next_words)
		probabilities_given_key = {}
		for unique_word in unique_words:
			probabilities_given_key[unique_word] = \
				float(next_words.count(unique_word)) / nb_words
		dictionnary[key] = probabilities_given_key

	return dictionnary

Get probabalities of next word

In [ ]:
def get_next_word(conditional_probabilities, current, next_candidate):

	if current in conditional_probabilities:
		if next_candidate in conditional_probabilities[current]:
			return conditional_probabilities[current][next_candidate]

	return 0

In [ ]:
remove_digits = str.maketrans('', '',string.digits)
remove_punctuation = str.maketrans('', '', string.punctuation)
corpus = corpus.translate(remove_digits)
corpus = corpus.translate(remove_punctuation)
tokens = list(set(word_tokenize(corpus)))

corpus

'at the nd annual grammy awards beyoncé received ten nominations including album of the year for i am sasha fierce record of the year for halo and song of the year for single ladies put a ring on it among others she tied with lauryn hill for most grammy nominations in a single year by a female artist in  beyoncé was featured on lady gagas single telephone and its music video the song topped the us pop songs chart becoming the sixth numberone for both beyoncé and gaga tying them with mariah carey for most numberones since the nielsen top  airplay chart launched in  telephone received a grammy award nomination for best pop collaboration with vocals'

In [ ]:
conditional_probabilities = build_conditional_probabilities(corpus)

In [ ]:
def check_sentence(s):
  prev = ""
  cum = 0
  for i in s.split():
    cum+=get_next_word(conditional_probabilities, prev, i)
    prev = i

  return cum

In [ ]:
#s1 = input("Enter a sentence - ")
#s2 = input("Enter a sentence again - ")

s1 = 'beyoncé received song of the year'
s2 = 'song of the year received beyoncé'

c1 = check_sentence(s1)
c2 = check_sentence(s2)

if c1>c2:
  print(f'"{s1}"', end=' ')
else:
  print(f'"{s2}"', end=' ')

print('IS more probabale')

"beyoncé received song of the year" IS more probabale


# Trigram

In [ ]:
text = corpus.split()

In [ ]:
trigrams = list(nltk.ngrams(text, 3, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
trigrams

[('<s>', '<s>', 'at'),
 ('<s>', 'at', 'the'),
 ('at', 'the', 'nd'),
 ('the', 'nd', 'annual'),
 ('nd', 'annual', 'grammy'),
 ('annual', 'grammy', 'awards'),
 ('grammy', 'awards', 'beyoncé'),
 ('awards', 'beyoncé', 'received'),
 ('beyoncé', 'received', 'ten'),
 ('received', 'ten', 'nominations'),
 ('ten', 'nominations', 'including'),
 ('nominations', 'including', 'album'),
 ('including', 'album', 'of'),
 ('album', 'of', 'the'),
 ('of', 'the', 'year'),
 ('the', 'year', 'for'),
 ('year', 'for', 'i'),
 ('for', 'i', 'am'),
 ('i', 'am', 'sasha'),
 ('am', 'sasha', 'fierce'),
 ('sasha', 'fierce', 'record'),
 ('fierce', 'record', 'of'),
 ('record', 'of', 'the'),
 ('of', 'the', 'year'),
 ('the', 'year', 'for'),
 ('year', 'for', 'halo'),
 ('for', 'halo', 'and'),
 ('halo', 'and', 'song'),
 ('and', 'song', 'of'),
 ('song', 'of', 'the'),
 ('of', 'the', 'year'),
 ('the', 'year', 'for'),
 ('year', 'for', 'single'),
 ('for', 'single', 'ladies'),
 ('single', 'ladies', 'put'),
 ('ladies', 'put', 'a'),
 ('

In [ ]:
cfdist = ConditionalFreqDist()

for w1, w2, w3 in trigrams:
  cfdist[(w1, w2)][w3] += 1

for w1_w2 in cfdist:
  total_count = float(sum(cfdist[w1_w2].values()))
  for w3 in cfdist[w1_w2]:
    cfdist[w1_w2][w3] /= total_count

In [ ]:
def predict(model, user_input):
    user_input = filter(user_input)
    user_input = user_input.split()

    w1 = len(user_input) - 2
    w2 = len(user_input)
    prev_words = user_input[w1:w2]

    # display prediction from highest to lowest maximum likelihood
    prediction = sorted(dict(model[prev_words[0], prev_words[1]]), key=lambda x: dict(model[prev_words[0], prev_words[1]])[x], reverse=True)
    print("Trigram model predictions: ", prediction)

    word = []
    weight = []
    for key, prob in dict(model[prev_words[0], prev_words[1]]).items():
        word.append(key)
        weight.append(prob)
    # pick from a weighted random probability of predictions
    next_word = random.choices(word, weights=weight, k=1)
    # add predicted word to user input
    user_input.append(next_word[0])
    print(' '.join(user_input))

    return ' '.join(user_input)

In [ ]:
def filter(text):
    # normalize text
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
    # replace html chars with ' '
    text = re.sub('<.*?>', ' ', text)
    # remove punctuation
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))
    # only alphabets and numerics
    text = re.sub('[^a-zA-Z]', ' ', text)
    # replace newline with space
    text = re.sub("\n", " ", text)
    # lower case
    text = text.lower()
    # split and join the words
    text = ' '.join(text.split())

    return text

In [ ]:
#s1 = input()
s1 = "beyoncé received song of the"
s2 = "beyoncé received song of the"

print("Simple Prediction: ", end = '')
predict(cfdist, s1)

print("Entire sentence Prediction: ", end = '')
s2 = predict(cfdist, s2)
s2 = predict(cfdist, s2)
s2 = predict(cfdist, s2)
s2 = predict(cfdist, s2)
s2 = predict(cfdist, s2)
s2 = predict(cfdist, s2)

Simple Prediction: Trigram model predictions:  ['year']
beyonce received song of the year
Entire sentence Prediction: Trigram model predictions:  ['year']
beyonce received song of the year
Trigram model predictions:  ['for']
beyonce received song of the year for
Trigram model predictions:  ['i', 'halo', 'single']
beyonce received song of the year for i
Trigram model predictions:  ['am']
beyonce received song of the year for i am
Trigram model predictions:  ['sasha']
beyonce received song of the year for i am sasha
Trigram model predictions:  ['fierce']
beyonce received song of the year for i am sasha fierce
